In [2]:
import csv
import scapy.all as sc
import pandas as pd
import numpy as np
import glob
import traceback
import logging
import re
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder
import gc
csv_dir = "D:\\tmp\\dataset\\UNSW-NB15\\csv_files"
preprocessed_csv_file = "D:\\tmp\\dataset\\UNSW-NB15\\preprocessed\preprocessed_csv_file.csv"
preprocessed_dir = "D:\\tmp\\dataset\\UNSW-NB15\\preprocessed\\"
pcap_dir = "E:\\tmp\\dataset\\UNSW-NB15\\raw_data\\17-2-2015"
combined_csv_out = "D:\\tmp\\dataset\\UNSW-NB15\\preprocessed\\combined_labelled_pcap_csv.csv"
final = preprocessed_dir + "/UNSW_final_cleaned_pcap.csv"

In [36]:
%xdel joblib
%whos

Variable             Type              Data/Info
------------------------------------------------
f                    BufferedWriter    <_io.BufferedWriter name=<...>\train_data_1500_1d.npy'>
gc                   module            <module 'gc' (built-in)>
np                   module            <module 'numpy' from 'C:\<...>ges\\numpy\\__init__.py'>
train_data_np        ndarray           2x1633542: 3267084 elems, type `object`, 26136672 bytes (24.925872802734375 Mb)
train_data_np_path   str               D:\tmp\dataset\UNSW-NB15\<...>ss\train_data_1500_1d.npy


In [40]:
dfs = []
for i in range(1, 5):
    path = csv_dir + f"/UNSW-NB15_{i}.csv"  
    dfs.append(pd.read_csv(path, header=None, low_memory=False))
all_data = pd.concat(dfs).reset_index(drop=True)

df_col = pd.read_csv(csv_dir + "/NUSW-NB15_features.csv", encoding="ISO-8859-1")
df_col["Name"] = df_col["Name"].apply(lambda x: x.strip().replace(" ", "").lower())
all_data.columns = df_col["Name"]
print(all_data.shape)

(2540047, 49)


In [61]:
all_data.isnull().sum()

Name
srcip                     0
sport                     0
dstip                     0
dsport                    0
proto                     0
state                     0
dur                       0
sbytes                    0
dbytes                    0
sttl                      0
dttl                      0
sloss                     0
dloss                     0
service                   0
sload                     0
dload                     0
spkts                     0
dpkts                     0
swin                      0
dwin                      0
stcpb                     0
dtcpb                     0
smeansz                   0
dmeansz                   0
trans_depth               0
res_bdy_len               0
sjit                      0
djit                      0
stime                     0
ltime                     0
sintpkt                   0
dintpkt                   0
tcprtt                    0
synack                    0
ackdat                    0
is_sm_ips_ports

In [41]:
all_data.drop(columns=["ct_flw_http_mthd", "is_ftp_login"], inplace=True)

In [42]:
all_data.isin([np.inf, -np.inf]).values.sum()


0

In [43]:
all_data["attack_cat"] = all_data.attack_cat.fillna(value="normal").apply(lambda x: x.strip().lower())
all_data["attack_cat"] = all_data["attack_cat"].replace("backdoors", "backdoor", regex=True).apply(lambda x: x.strip().lower())

In [44]:
all_data.isnull().values.any()


False

In [66]:
all_data["attack_cat"].value_counts()

normal            2218764
generic            215481
exploits            44525
fuzzers             24246
dos                 16353
reconnaissance      13987
analysis             2677
backdoor             2329
shellcode            1511
worms                 174
Name: attack_cat, dtype: int64

In [67]:
all_data.ct_ftp_cmd.unique()

array([0, 1, 6, 2, 4, 8, 5, 3, '0', '1', '4', '2', ' '], dtype=object)

In [46]:
all_data["ct_ftp_cmd"] = all_data["ct_ftp_cmd"].apply(lambda x: 0 if x == " " else x).astype(int)

In [69]:
all_data.shape

(2540047, 47)

In [47]:
df = all_data[all_data.duplicated(keep=False)]

In [48]:
df.shape

(568841, 47)

In [49]:
all_data.drop_duplicates(inplace=True)
all_data.shape

(2059415, 47)

In [50]:
len(all_data.proto.unique())

135

In [51]:
pd.set_option("display.max_rows", None)
Important_protocol = all_data.proto[all_data.label == 1].value_counts()
print(Important_protocol)
pd.reset_option("display.max_rows")

tcp            54197
udp            33114
unas            4765
ospf             993
sctp             444
any              138
gre               95
rsvp              92
ipv6              85
sep               81
sun-nd            81
swipe             81
mobile            81
pim               81
nvp               51
snp               51
secure-vmtp       51
emcon             51
vmtp              51
etherip           46
encap             46
ax.25             46
aes-sp3-d         46
micp              46
pri-enc           46
gmtp              46
ipip              46
nsfnet-igp        46
mtp               46
larp              46
sprite-rpc        46
eigrp             46
tcf               46
dgp               46
ifmp              46
ttp               46
vines             46
iso-ip            46
wb-expak          46
wb-mon            46
pnni              46
ipcomp            46
aris              46
smp               46
fc                46
iplt              46
pipe              46
sps          

In [52]:
b = Important_protocol.keys()[25:]
b = b.drop("gmtp")
b = b.drop("ipip")
b = b.drop("larp")
b = b.drop("dgp")
b = b.drop("pnni")
b = b.drop("fc")
b = b.drop("iplt")
b = b.drop("pipe")
b = b.drop("sps")
b = b.drop("sccopmce")
b = b.drop("crudp")
b = b.drop("crtp")
b = b.drop("fire")
b = b.drop("rvd")
b = b.drop("rdp")
b = b.drop("hmp")
b = b.drop("pup")
b = b.drop("egp")
b = b.drop("ip")
b = b.drop("ib")
b

Index(['nsfnet-igp', 'mtp', 'sprite-rpc', 'eigrp', 'tcf', 'ifmp', 'ttp',
       'vines', 'iso-ip', 'wb-expak', 'wb-mon', 'ipcomp', 'aris', 'smp',
       'isis', 'ptp', 'sm', 'uti', 'stp', 'qnx', 'srp', 'l2tp', 'ddx', 'iatp',
       'pgm', 'zero', 'vrrp', 'ipx-n-ip', 'compaq-peer', 'pvp', 'scps', 'a/n',
       'br-sat-mon', 'wsn', 'merit-inp', 'netblt', 'iso-tp4', 'irtp', 'leaf-2',
       'leaf-1', 'trunk-2', 'xns-idp', 'trunk-1', 'prm', 'dcn', 'mux', 'chaos',
       'bbn-rcc', 'argus', 'xnet', 'igp', 'cbt', 'st2', 'ggp', 'ipnip',
       'mfe-nsp', '3pc', 'cphb', 'xtp', 'cpnx', 'visa', 'ipcv', 'sat-mon',
       'ippc', 'kryptolan', 'sat-expak', 'cftp', 'ipv6-opts', 'skip', 'tlsp',
       'ipv6-no', 'narp', 'i-nlsp', 'bna', 'mhrp', 'ipv6-route', 'sdrp',
       'ipv6-frag', 'idrp', 'il', 'idpr-cmtp', 'ddp', 'tp++', 'idpr'],
      dtype='object')

In [53]:
all_data["proto"].replace(b, "others", inplace=True)

In [54]:
all_data.proto.value_counts()

tcp            1448859
udp             588026
arp               6658
unas              4765
ospf              3964
others            3864
icmp               498
sctp               444
any                138
gre                 95
rsvp                92
ipv6                85
sep                 81
swipe               81
sun-nd              81
mobile              81
pim                 81
igmp                54
emcon               51
nvp                 51
snp                 51
vmtp                51
secure-vmtp         51
fire                46
pri-enc             46
gmtp                46
pnni                46
crudp               46
crtp                46
iplt                46
fc                  46
encap               46
sccopmce            46
sps                 46
pipe                46
etherip             46
dgp                 46
aes-sp3-d           46
micp                46
ipip                46
ax.25               46
larp                46
rvd                 46
rdp        

In [55]:
all_data.to_csv(preprocessed_csv_file, index=False)

In [ ]:
all_data = pd.read_csv(preprocessed_csv_file)

In [28]:
selected = ["srcip", "sport", "dstip", "dsport", "attack_cat"]
selected_data = all_data[selected]
selected_data.drop_duplicates(inplace=True)
pks = ["srcip", "sport", "dstip", "dsport"]
key_feature_value = selected_data[pks]
print(len(key_feature_value))
key_feature_value.drop_duplicates(inplace=True)
print(len(key_feature_value))

1965544
1964996


In [94]:
def numeric_ordering(file):
    match = re.findall(r"\d+", file)
    if not match:
        return math.inf
    return int(match[-1])

In [95]:
pcap_file_list = sorted(glob.glob(pcap_dir + "/*.pcap"), key=numeric_ordering)
print(pcap_file_list)

['E:\\tmp\\dataset\\UNSW-NB15\\raw_data\\17-2-2015\\1.pcap', 'E:\\tmp\\dataset\\UNSW-NB15\\raw_data\\17-2-2015\\2.pcap', 'E:\\tmp\\dataset\\UNSW-NB15\\raw_data\\17-2-2015\\3.pcap', 'E:\\tmp\\dataset\\UNSW-NB15\\raw_data\\17-2-2015\\4.pcap', 'E:\\tmp\\dataset\\UNSW-NB15\\raw_data\\17-2-2015\\5.pcap', 'E:\\tmp\\dataset\\UNSW-NB15\\raw_data\\17-2-2015\\6.pcap', 'E:\\tmp\\dataset\\UNSW-NB15\\raw_data\\17-2-2015\\7.pcap', 'E:\\tmp\\dataset\\UNSW-NB15\\raw_data\\17-2-2015\\8.pcap', 'E:\\tmp\\dataset\\UNSW-NB15\\raw_data\\17-2-2015\\9.pcap', 'E:\\tmp\\dataset\\UNSW-NB15\\raw_data\\17-2-2015\\10.pcap', 'E:\\tmp\\dataset\\UNSW-NB15\\raw_data\\17-2-2015\\11.pcap', 'E:\\tmp\\dataset\\UNSW-NB15\\raw_data\\17-2-2015\\12.pcap', 'E:\\tmp\\dataset\\UNSW-NB15\\raw_data\\17-2-2015\\13.pcap', 'E:\\tmp\\dataset\\UNSW-NB15\\raw_data\\17-2-2015\\14.pcap', 'E:\\tmp\\dataset\\UNSW-NB15\\raw_data\\17-2-2015\\15.pcap', 'E:\\tmp\\dataset\\UNSW-NB15\\raw_data\\17-2-2015\\16.pcap', 'E:\\tmp\\dataset\\UNSW-NB15\\ra

In [93]:
def pcap_parser(pcap_file_list, file_num):
    logging.basicConfig(level=logging.DEBUG, format="%(asctime)s - %(levelname)s -%(message)s")
    packet_counter = 0
    skipped_counter = 0
    all_rows = []
    for pcap_file in pcap_file_list:
        logging.info("Reading input file:  %s", pcap_file)
        packet_counter = 0
        skipped_counter = 0
        all_rows = []
        pcap = sc.PcapReader(pcap_file)
        while True:
            try:
                f = pcap.read_packet()
                if sc.IP in f:
                    i2s = {
                        0: "ip",
                        1: "icmp",
                        2: "igmp",
                        3: "ggp",
                        6: "tcp",
                        8: "egp",
                        11: "nvp",
                        12: "pup",
                        14: "emcon",
                        17: "udp",
                        20: "hmp",
                        27: "rdp",
                        33: "sep",
                        41: "ipv6",
                        46: "rsvp",
                        47: "gre",
                        53: "swipe",
                        55: "mobile",
                        61: "any",
                        63: "any",
                        66: "rvd",
                        68: "any",
                        77: "sun-nd",
                        81: "vmtp",
                        82: "secure-vmtp",
                        86: "dgp",
                        89: "ospf",
                        91: "larp",
                        93: "ax.25",
                        94: "ipip",
                        95: "micp",
                        96: "aes-sp3-d",
                        97: "etherip",
                        98: "encap",
                        99: "any",
                        100: "gmtp",
                        102: "pnni",
                        103: "pim",
                        109: "snp",
                        125: "fire",
                        126: "crtp",
                        127: "crudp",
                        128: "sccopmce",
                        129: "iplt",
                        130: "sps",
                        131: "pipe",
                        132: "sctp",
                        135: "fc",
                        253: "ib",
                        255: "unas",
                    }
                    if f[1].proto in i2s:
                        proto_m = i2s[f[1].proto]
                    else:
                        proto_m = "others"
                    src_ip = f["IP"].src
                    dst_ip = f["IP"].dst
                    total_len = f["IP"].len  # Total Length
                    sttl = f["IP"].ttl
                    epoch = f.time
                    payload_protocols = [sc.UDP, sc.TCP, sc.ICMP]
                    payload_protocol = next((element for element in payload_protocols if element in f), None)
                    try:
                        if payload_protocol:
                            payload = bytes(f[payload_protocol].payload).hex()
                        else:
                            payload = bytes(f.load).hex()
                    except AttributeError:
                        payload = "0"
                    try:
                        sport = f.sport
                        dport = f.dport
                    except AttributeError:
                        sport = 0
                        dport = 0
                    if proto_m == "sctp":
                        sport = 0
                        dport = 0
                elif f.haslayer(sc.ARP):
                    src_ip = f["ARP"].psrc 
                    dst_ip = f["ARP"].pdst 
                    epoch = f.time
                    if len(f.layers()) > 2:
                        payload =f.getlayer(sc.ARP).load.hex()
                    else:
                        payload = 0
                    sttl = 0
                    sport = 0
                    dport = 0
                    total_len = 0
                else:
                    skipped_counter += 1
                    print(f.show())
                    continue
                all_rows.append([packet_counter, epoch, src_ip, sport, dst_ip, dport, proto_m, sttl, total_len, payload])
                packet_counter += 1
                if packet_counter % 100000 == 0:
                    logging.info(f"Done with {packet_counter} Packets")
            except EOFError:
                logging.info(f"Done reading {pcap_file}")
                break
            except Exception as e:
                print(f"Unknown Exception during parsing of {pcap_file}, aborting parsing: ", e)
                traceback.print_exc()
                return
        out = pd.DataFrame(
            all_rows,
            columns=[
                "frame_num",
                "stime",
                "srcip",
                "sport",
                "dstip",
                "dsport",
                "protocol_m",
                "sttl",
                "total_len",
                "payload",
            ],
        )
        out["stime"] = out["stime"].astype(float).round().astype(int)
        logging.info(f"Skipped {skipped_counter} unparsable packets.")
        logging.info("Exporting CSV File#%s", file_num)
        csv_file = preprocessed_dir + "pcap_csv_" + str(file_num) + ".csv"
        out.to_csv(csv_file, index=False)
        file_num += 1

In [97]:
pcap_parser(pcap_file_list, 1)

2023-06-08 20:37:43,223 - INFO -Reading input file:  E:\tmp\dataset\UNSW-NB15\raw_data\17-2-2015\1.pcap
2023-06-08 20:38:08,383 - INFO -Done with 100000 Packets
2023-06-08 20:38:33,668 - INFO -Done with 200000 Packets
2023-06-08 20:39:01,008 - INFO -Done with 300000 Packets
2023-06-08 20:39:26,698 - INFO -Done with 400000 Packets
2023-06-08 20:39:52,401 - INFO -Done with 500000 Packets
2023-06-08 20:40:18,172 - INFO -Done with 600000 Packets
2023-06-08 20:40:43,933 - INFO -Done with 700000 Packets
2023-06-08 20:41:12,253 - INFO -Done with 800000 Packets
2023-06-08 20:41:38,601 - INFO -Done with 900000 Packets
2023-06-08 20:42:04,759 - INFO -Done with 1000000 Packets
2023-06-08 20:42:31,314 - INFO -Done with 1100000 Packets
2023-06-08 20:42:57,204 - INFO -Done with 1200000 Packets
2023-06-08 20:43:23,293 - INFO -Done with 1300000 Packets
2023-06-08 20:43:48,028 - WARNING -DNS decompression loop detected
2023-06-08 20:43:48,034 - WARNING -DNS decompression loop detected
2023-06-08 20:43:

2023-06-08 21:30:46,604 - INFO -Done with 600000 Packets
2023-06-08 21:31:12,655 - INFO -Done with 700000 Packets
2023-06-08 21:31:38,591 - INFO -Done with 800000 Packets
2023-06-08 21:32:04,726 - INFO -Done with 900000 Packets
2023-06-08 21:32:30,541 - INFO -Done with 1000000 Packets
2023-06-08 21:32:56,264 - INFO -Done with 1100000 Packets
2023-06-08 21:33:24,869 - INFO -Done with 1200000 Packets
2023-06-08 21:33:50,651 - INFO -Done with 1300000 Packets
2023-06-08 21:34:16,476 - INFO -Done with 1400000 Packets
2023-06-08 21:34:41,898 - INFO -Done with 1500000 Packets
2023-06-08 21:35:07,580 - INFO -Done with 1600000 Packets
2023-06-08 21:35:16,679 - WARNING -DNS decompression loop detected
2023-06-08 21:35:16,692 - WARNING -DNS decompression loop detected
2023-06-08 21:35:23,681 - WARNING -DNS decompression loop detected
2023-06-08 21:35:23,691 - WARNING -DNS decompression loop detected
2023-06-08 21:35:33,485 - INFO -Done with 1700000 Packets
2023-06-08 21:35:59,160 - INFO -Done wit

2023-06-08 22:23:47,831 - INFO -Done with 700000 Packets
2023-06-08 22:24:14,660 - INFO -Done with 800000 Packets
2023-06-08 22:24:43,994 - INFO -Done with 900000 Packets
2023-06-08 22:25:10,768 - INFO -Done with 1000000 Packets
2023-06-08 22:25:37,107 - INFO -Done with 1100000 Packets
2023-06-08 22:26:03,362 - INFO -Done with 1200000 Packets
2023-06-08 22:26:29,871 - INFO -Done with 1300000 Packets
2023-06-08 22:26:56,332 - INFO -Done with 1400000 Packets
2023-06-08 22:27:23,043 - INFO -Done with 1500000 Packets
2023-06-08 22:27:52,827 - INFO -Done with 1600000 Packets
2023-06-08 22:28:19,265 - INFO -Done with 1700000 Packets
2023-06-08 22:28:45,727 - INFO -Done with 1800000 Packets
2023-06-08 22:29:00,163 - WARNING -DNS decompression loop detected
2023-06-08 22:29:00,176 - WARNING -DNS decompression loop detected
2023-06-08 22:29:12,340 - INFO -Done with 1900000 Packets
2023-06-08 22:29:39,232 - INFO -Done with 2000000 Packets
2023-06-08 22:30:06,051 - INFO -Done with 2100000 Packets

2023-06-08 23:17:20,825 - INFO -Done with 900000 Packets
2023-06-08 23:17:47,707 - INFO -Done with 1000000 Packets
2023-06-08 23:18:14,378 - INFO -Done with 1100000 Packets
2023-06-08 23:18:40,812 - INFO -Done with 1200000 Packets
2023-06-08 23:19:07,277 - INFO -Done with 1300000 Packets
2023-06-08 23:19:33,694 - INFO -Done with 1400000 Packets
2023-06-08 23:20:03,402 - INFO -Done with 1500000 Packets
2023-06-08 23:20:29,872 - INFO -Done with 1600000 Packets
2023-06-08 23:20:56,203 - INFO -Done with 1700000 Packets
2023-06-08 23:21:22,426 - INFO -Done with 1800000 Packets
2023-06-08 23:21:48,770 - INFO -Done with 1900000 Packets
2023-06-08 23:22:15,611 - INFO -Done with 2000000 Packets
2023-06-08 23:22:42,448 - INFO -Done with 2100000 Packets
2023-06-08 23:23:09,232 - INFO -Done with 2200000 Packets
2023-06-08 23:23:39,690 - INFO -Done with 2300000 Packets
2023-06-08 23:24:06,009 - INFO -Done with 2400000 Packets
2023-06-08 23:24:32,068 - INFO -Done with 2500000 Packets
2023-06-08 23:2

2023-06-09 00:11:22,742 - INFO -Done with 1600000 Packets
2023-06-09 00:11:48,444 - INFO -Done with 1700000 Packets
2023-06-09 00:12:14,883 - INFO -Done with 1800000 Packets
2023-06-09 00:12:41,288 - INFO -Done with 1900000 Packets
2023-06-09 00:13:07,701 - INFO -Done with 2000000 Packets
2023-06-09 00:13:33,812 - INFO -Done with 2100000 Packets
2023-06-09 00:14:03,703 - INFO -Done with 2200000 Packets
2023-06-09 00:14:30,414 - INFO -Done with 2300000 Packets
2023-06-09 00:14:57,177 - INFO -Done with 2400000 Packets
2023-06-09 00:15:23,954 - INFO -Done with 2500000 Packets
2023-06-09 00:15:50,883 - INFO -Done with 2600000 Packets
2023-06-09 00:16:17,861 - INFO -Done with 2700000 Packets
2023-06-09 00:16:45,095 - INFO -Done with 2800000 Packets
2023-06-09 00:17:11,533 - INFO -Done with 2900000 Packets
2023-06-09 00:17:38,064 - INFO -Done with 3000000 Packets
2023-06-09 00:18:04,408 - INFO -Done with 3100000 Packets
2023-06-09 00:18:35,030 - INFO -Done with 3200000 Packets
2023-06-09 00:

2023-06-09 01:06:24,820 - INFO -Done with 2300000 Packets
2023-06-09 01:06:51,951 - INFO -Done with 2400000 Packets
2023-06-09 01:07:19,205 - INFO -Done with 2500000 Packets
2023-06-09 01:07:46,170 - INFO -Done with 2600000 Packets
2023-06-09 01:08:13,015 - INFO -Done with 2700000 Packets
2023-06-09 01:08:39,456 - INFO -Done with 2800000 Packets
2023-06-09 01:09:05,634 - INFO -Done with 2900000 Packets
2023-06-09 01:09:31,928 - INFO -Done with 3000000 Packets
2023-06-09 01:09:58,233 - INFO -Done with 3100000 Packets
2023-06-09 01:10:29,266 - INFO -Done with 3200000 Packets
2023-06-09 01:10:55,653 - INFO -Done with 3300000 Packets
2023-06-09 01:11:21,963 - INFO -Done with 3400000 Packets
2023-06-09 01:11:48,555 - INFO -Done with 3500000 Packets
2023-06-09 01:12:16,373 - INFO -Done with 3600000 Packets
2023-06-09 01:12:16,822 - INFO -Done reading E:\tmp\dataset\UNSW-NB15\raw_data\17-2-2015\16.pcap
2023-06-09 01:12:25,627 - INFO -Skipped 0 unparsable packets.
2023-06-09 01:12:25,637 - INF

2023-06-09 02:01:13,312 - INFO -Done with 3300000 Packets
2023-06-09 02:01:44,648 - INFO -Done with 3400000 Packets
2023-06-09 02:01:46,964 - INFO -Done reading E:\tmp\dataset\UNSW-NB15\raw_data\17-2-2015\19.pcap
2023-06-09 02:01:55,432 - INFO -Skipped 0 unparsable packets.
2023-06-09 02:01:55,436 - INFO -Exporting CSV File#19
2023-06-09 02:03:07,673 - INFO -Reading input file:  E:\tmp\dataset\UNSW-NB15\raw_data\17-2-2015\20.pcap
2023-06-09 02:03:34,928 - INFO -Done with 100000 Packets
2023-06-09 02:04:01,355 - INFO -Done with 200000 Packets
2023-06-09 02:04:27,899 - INFO -Done with 300000 Packets
2023-06-09 02:04:54,336 - INFO -Done with 400000 Packets
2023-06-09 02:05:20,999 - INFO -Done with 500000 Packets
2023-06-09 02:05:47,590 - INFO -Done with 600000 Packets
2023-06-09 02:06:13,832 - WARNING -DNS decompression loop detected
2023-06-09 02:06:13,852 - WARNING -DNS decompression loop detected
2023-06-09 02:06:14,118 - INFO -Done with 700000 Packets
2023-06-09 02:06:40,851 - INFO -D

2023-06-09 02:57:05,136 - INFO -Reading input file:  E:\tmp\dataset\UNSW-NB15\raw_data\17-2-2015\23.pcap
2023-06-09 02:57:32,188 - INFO -Done with 100000 Packets
2023-06-09 02:57:58,008 - INFO -Done with 200000 Packets
2023-06-09 02:58:24,098 - INFO -Done with 300000 Packets
2023-06-09 02:58:50,236 - INFO -Done with 400000 Packets
2023-06-09 02:59:20,121 - INFO -Done with 500000 Packets
2023-06-09 02:59:46,089 - INFO -Done with 600000 Packets
2023-06-09 03:00:12,057 - INFO -Done with 700000 Packets
2023-06-09 03:00:38,250 - INFO -Done with 800000 Packets
2023-06-09 03:01:04,487 - INFO -Done with 900000 Packets
2023-06-09 03:01:30,691 - INFO -Done with 1000000 Packets
2023-06-09 03:01:59,867 - INFO -Done with 1100000 Packets
2023-06-09 03:02:25,973 - INFO -Done with 1200000 Packets
2023-06-09 03:02:52,133 - INFO -Done with 1300000 Packets
2023-06-09 03:03:18,546 - INFO -Done with 1400000 Packets
2023-06-09 03:03:38,409 - WARNING -DNS decompression loop detected
2023-06-09 03:03:38,436 -

2023-06-09 03:49:39,961 - INFO -Done with 700000 Packets
2023-06-09 03:50:04,477 - INFO -Done with 800000 Packets
2023-06-09 03:50:29,134 - INFO -Done with 900000 Packets
2023-06-09 03:50:53,799 - INFO -Done with 1000000 Packets
2023-06-09 03:51:18,466 - INFO -Done with 1100000 Packets
2023-06-09 03:51:43,347 - INFO -Done with 1200000 Packets
2023-06-09 03:52:10,937 - INFO -Done with 1300000 Packets
2023-06-09 03:52:35,794 - INFO -Done with 1400000 Packets
2023-06-09 03:53:00,466 - INFO -Done with 1500000 Packets
2023-06-09 03:53:25,100 - INFO -Done with 1600000 Packets
2023-06-09 03:53:49,807 - INFO -Done with 1700000 Packets
2023-06-09 03:54:14,590 - INFO -Done with 1800000 Packets
2023-06-09 03:54:16,801 - WARNING -DNS decompression loop detected
2023-06-09 03:54:16,814 - WARNING -DNS decompression loop detected
2023-06-09 03:54:22,282 - WARNING -DNS decompression loop detected
2023-06-09 03:54:22,293 - WARNING -DNS decompression loop detected
2023-06-09 03:54:39,327 - INFO -Done wi

In [101]:
df = pd.read_csv("D:\\tmp\\dataset\\UNSW-NB15\\preprocessed\\pcap_csv_1.csv")

In [120]:
def label_UNSW(pcap_csv, file_num):
    logging.info("Reading Pre-processed UNSW CSV_file...")
    df_flow = pd.read_csv(preprocessed_csv_file, low_memory=False)
    df_flow = df_flow[["stime", "ltime", "dur", "srcip", "dstip", "dsport", "sport", "sttl", "proto", "attack_cat", "label"]]
    df_flow.rename(columns={"proto": "protocol_m"}, inplace=True)
    df_flow["ltime2"] = (df_flow.stime + df_flow.dur).round().astype("int32")
    df_flow["ltime_max"] = df_flow[["ltime", "ltime2"]].max(axis=1)
    df_flow.loc[:, "dsport"] = df_flow.dsport.apply(lambda x: int(x, base=16) if x.startswith("0x") else x)
    df_flow.loc[:, "sport"] = df_flow.sport.apply(lambda x: int(x, base=16) if x.startswith("0x") else x)
    d = pd.to_numeric(df_flow.dsport, errors="coerce")
    s = pd.to_numeric(df_flow.sport, errors="coerce")
    d[d.isna()] = 0
    s[s.isna()] = 0
    df_flow.dsport = d
    df_flow.sport = s
    df_flow.loc[df_flow.protocol_m == "icmp", ["sport", "dsport"]] = 0

    for pcap_file in pcap_csv:
        logging.info("Reading Parsed_Pcap_file_%s...", file_num)
        df_pcap_csv = pd.read_csv(pcap_file, index_col=0, low_memory=False)

        combine1 = pd.merge(df_pcap_csv, df_flow, how="left", on=["srcip", "dstip", "dsport", "sport", "protocol_m"], suffixes=["", "_flow"])
        combine2 = pd.merge(
            df_pcap_csv,
            df_flow,
            how="left",
            left_on=["srcip", "dstip", "dsport", "sport", "protocol_m"],
            right_on=["dstip", "srcip", "sport", "dsport", "protocol_m"],
            suffixes=["", "_flow"],
        )
        combine = pd.concat([combine1, combine2])
        combine.drop(columns=["sttl_flow", "dur", "srcip_flow", "dstip_flow", "dsport_flow", "sport_flow", "ltime"], inplace=True)
        combine = combine[~combine.label.isna()]
        combine = combine[(combine["stime_flow"] <= combine["stime"]) & (combine["stime"] <= combine["ltime_max"])]
        selected = ["payload", "sttl", "total_len", "protocol_m", "attack_cat"]
        combine = combine[selected]
        combine.drop_duplicates(inplace=True)
        pks1 = ["payload", "sttl", "total_len", "protocol_m"]
        key_feature_value = combine[pks1]
        key_feature_value.drop_duplicates(inplace=True)
        combine = combine.loc[key_feature_value.index] 
        csv_out = preprocessed_dir + "labelled_pcap_csv_" + str(file_num) + ".csv"
        combine.to_csv(csv_out, index=False)
        file_num += 1

In [122]:
pcap_csv = sorted(glob.glob(preprocessed_dir + "/pcap_csv_*.csv"), key=numeric_ordering)
print(pcap_csv)
label_UNSW(pcap_csv, 1)

['D:\\tmp\\dataset\\UNSW-NB15\\preprocessed\\pcap_csv_1.csv', 'D:\\tmp\\dataset\\UNSW-NB15\\preprocessed\\pcap_csv_2.csv', 'D:\\tmp\\dataset\\UNSW-NB15\\preprocessed\\pcap_csv_3.csv', 'D:\\tmp\\dataset\\UNSW-NB15\\preprocessed\\pcap_csv_4.csv', 'D:\\tmp\\dataset\\UNSW-NB15\\preprocessed\\pcap_csv_5.csv', 'D:\\tmp\\dataset\\UNSW-NB15\\preprocessed\\pcap_csv_6.csv', 'D:\\tmp\\dataset\\UNSW-NB15\\preprocessed\\pcap_csv_7.csv', 'D:\\tmp\\dataset\\UNSW-NB15\\preprocessed\\pcap_csv_8.csv', 'D:\\tmp\\dataset\\UNSW-NB15\\preprocessed\\pcap_csv_9.csv', 'D:\\tmp\\dataset\\UNSW-NB15\\preprocessed\\pcap_csv_10.csv', 'D:\\tmp\\dataset\\UNSW-NB15\\preprocessed\\pcap_csv_11.csv', 'D:\\tmp\\dataset\\UNSW-NB15\\preprocessed\\pcap_csv_12.csv', 'D:\\tmp\\dataset\\UNSW-NB15\\preprocessed\\pcap_csv_13.csv', 'D:\\tmp\\dataset\\UNSW-NB15\\preprocessed\\pcap_csv_14.csv', 'D:\\tmp\\dataset\\UNSW-NB15\\preprocessed\\pcap_csv_15.csv', 'D:\\tmp\\dataset\\UNSW-NB15\\preprocessed\\pcap_csv_16.csv', 'D:\\tmp\\datase

2023-06-09 11:45:16,018 - INFO -Reading Pre-processed UNSW CSV_file...
2023-06-09 11:45:27,539 - INFO -Reading Parsed_Pcap_file_1...
2023-06-09 11:46:33,431 - INFO -Reading Parsed_Pcap_file_2...
2023-06-09 11:47:21,303 - INFO -Reading Parsed_Pcap_file_3...
2023-06-09 11:48:12,864 - INFO -Reading Parsed_Pcap_file_4...
2023-06-09 11:49:04,571 - INFO -Reading Parsed_Pcap_file_5...
2023-06-09 11:49:53,250 - INFO -Reading Parsed_Pcap_file_6...
2023-06-09 11:50:39,400 - INFO -Reading Parsed_Pcap_file_7...
2023-06-09 11:51:28,810 - INFO -Reading Parsed_Pcap_file_8...
2023-06-09 11:52:19,353 - INFO -Reading Parsed_Pcap_file_9...
2023-06-09 11:53:09,694 - INFO -Reading Parsed_Pcap_file_10...
2023-06-09 11:53:57,554 - INFO -Reading Parsed_Pcap_file_11...
2023-06-09 11:54:47,536 - INFO -Reading Parsed_Pcap_file_12...
2023-06-09 11:55:42,208 - INFO -Reading Parsed_Pcap_file_13...
2023-06-09 11:56:31,177 - INFO -Reading Parsed_Pcap_file_14...
2023-06-09 11:57:19,030 - INFO -Reading Parsed_Pcap_file

In [123]:
def combine_UNSW(in_file_path):
    combine = pd.DataFrame()
    for files in in_file_path:
        df = pd.read_csv(files)
        combine = pd.concat([combine, df], axis=0, ignore_index=True)
        print(combine.shape)
    logging.info("Exporting_combined_csv_file....")
    combine.to_csv(combined_csv_out, index=False)
    return combine

In [125]:
in_file = sorted(glob.glob(preprocessed_dir + "labelled_pcap_csv_*.csv"), key=numeric_ordering)
print(in_file)
df_payload = combine_UNSW(in_file)
logging.info("Total Shape of Combined Data Before Processing is: %s", df_payload.shape)
logging.info("Removing Duplicates ......")
df_payload.drop_duplicates(inplace=True)
logging.info("Removing Non-Payload Data Instances (e.g. payload is all zero) .........")
df_payload.drop(df_payload[df_payload.payload.isnull()].index, inplace=True)
df_payload = df_payload[~df_payload.payload.str.fullmatch("0+")]
df_payload.drop_duplicates(inplace=True)
pks1 = ["payload", "sttl", "total_len", "protocol_m"]
key_feature_value = df_payload[pks1]
key_feature_value.drop_duplicates(inplace=True)
df_payload = df_payload.loc[key_feature_value.index]
logging.info("Total Shape of Combined Data After Processing is: %s", df_payload.shape)
logging.info("Exporting Finalized Version of Data .............")
df_payload.to_csv(final, index=False)
logging.info("Process Completed ...................")
logging.info(f"Final output CSV saved at {final}")

['D:\\tmp\\dataset\\UNSW-NB15\\preprocessed\\labelled_pcap_csv_1.csv', 'D:\\tmp\\dataset\\UNSW-NB15\\preprocessed\\labelled_pcap_csv_2.csv', 'D:\\tmp\\dataset\\UNSW-NB15\\preprocessed\\labelled_pcap_csv_3.csv', 'D:\\tmp\\dataset\\UNSW-NB15\\preprocessed\\labelled_pcap_csv_4.csv', 'D:\\tmp\\dataset\\UNSW-NB15\\preprocessed\\labelled_pcap_csv_5.csv', 'D:\\tmp\\dataset\\UNSW-NB15\\preprocessed\\labelled_pcap_csv_6.csv', 'D:\\tmp\\dataset\\UNSW-NB15\\preprocessed\\labelled_pcap_csv_7.csv', 'D:\\tmp\\dataset\\UNSW-NB15\\preprocessed\\labelled_pcap_csv_8.csv', 'D:\\tmp\\dataset\\UNSW-NB15\\preprocessed\\labelled_pcap_csv_9.csv', 'D:\\tmp\\dataset\\UNSW-NB15\\preprocessed\\labelled_pcap_csv_10.csv', 'D:\\tmp\\dataset\\UNSW-NB15\\preprocessed\\labelled_pcap_csv_11.csv', 'D:\\tmp\\dataset\\UNSW-NB15\\preprocessed\\labelled_pcap_csv_12.csv', 'D:\\tmp\\dataset\\UNSW-NB15\\preprocessed\\labelled_pcap_csv_13.csv', 'D:\\tmp\\dataset\\UNSW-NB15\\preprocessed\\labelled_pcap_csv_14.csv', 'D:\\tmp\\data

2023-06-09 12:11:09,809 - INFO -Exporting_combined_csv_file....
2023-06-09 12:13:25,111 - INFO -Total Shape of Combined Data Before Processing is: (4029051, 5)
2023-06-09 12:13:25,115 - INFO -Removing Duplicates ......
2023-06-09 12:13:28,977 - INFO -Removing Non-Payload Data Instances (e.g. payload is all zero) .........
2023-06-09 12:14:02,503 - INFO -Total Shape of Combined Data After Processing is: (3444441, 5)


In [3]:
data = pd.read_csv(final)

In [4]:
data['attack_cat'].value_counts()

normal            1688060
exploits          1188782
dos                332831
generic            101826
fuzzers             89594
reconnaissance      31312
worms                7682
analysis             2428
backdoor             1259
shellcode             667
Name: attack_cat, dtype: int64

In [3]:
count_dict = dict(data['attack_cat'].value_counts())
data.drop(data[data['attack_cat'].isin(['exploits'])].sample(count_dict['exploits'] - int(1.5*count_dict['dos'])).index, inplace=True)
data.drop(data[data['attack_cat'].isin(['normal'])].sample(count_dict['normal'] - int(1.5*1.5*count_dict['dos'])).index, inplace=True)
data['attack_cat'].value_counts()
data.reset_index(drop=True, inplace=True)

data = data[~data['attack_cat'].isin(['shellcode'])]
data.reset_index(drop=True, inplace=True)

protocolencoder = LabelEncoder()
data["protocol_m_label"]=protocolencoder.fit_transform(data.protocol_m.values)
data.drop(columns=["protocol_m"], inplace=True)

In [6]:
train_data_path = "D:\\tmp\\dataset\\UNSW-NB15\\preprocessed\\downsampled\\9class_v2\\train_data.csv"
test_data_path = "D:\\tmp\\dataset\\UNSW-NB15\\preprocessed\\downsampled\\9class_v2\\test_data.csv"

# train_data_path = "D:\\tmp\\dataset\\UNSW-NB15\\preprocessed\\downsampled\\9class_v1\\train_data.csv"
# test_data_path = "D:\\tmp\\dataset\\UNSW-NB15\\preprocessed\\downsampled\\9class_v1\\test_data.csv"

# test_data_np_path = "D:\\tmp\\dataset\\UNSW-NB15\\preprocessed\\downsampled\\784_2d_9class\\test_data_784_2d.npy"
# train_data_np_path = "D:\\tmp\\dataset\\UNSW-NB15\\preprocessed\\downsampled\\784_2d_9class\\train_data_784_2d.npy"

test_data_np_path = "D:\\tmp\\dataset\\UNSW-NB15\\preprocessed\\downsampled\\1500_1d_9class\\test_data_1500_1d.npy"
train_data_np_path = "D:\\tmp\\dataset\\UNSW-NB15\\preprocessed\\downsampled\\1500_1d_9class\\train_data_1500_1d"

# test_data_np_path = "D:\\tmp\\dataset\\UNSW-NB15\\preprocessed\\downsampled\\50_1d_9class\\test_data_50_1d.npy"
# train_data_np_path = "D:\\tmp\\dataset\\UNSW-NB15\\preprocessed\\downsampled\\50_1d_9class\\train_data_50_1d"

# test_data_np_path = "D:\\tmp\\dataset\\UNSW-NB15\\preprocessed\\downsampled\\784_1d_9class\\test_data_784_1d.npy"
# train_data_np_path = "D:\\tmp\\dataset\\UNSW-NB15\\preprocessed\\downsampled\\784_1d_9class\\train_data_784_1d"

# test_data_np_path = "D:\\tmp\\dataset\\UNSW-NB15\\preprocessed\\downsampled\\5000_1d_9class\\test_data_5000_1d.npy"
# train_data_np_path = "D:\\tmp\\dataset\\UNSW-NB15\\preprocessed\\downsampled\\5000_1d_9class\\train_data_5000_1d"

In [5]:
def build_representation(row):
    return f'{row[1]:x}' + f'{row[2]:0>4x}' + f'{row[4]:x}' + row[0]

representations = [build_representation(i) for i in data.values]
data["representations"] = representations
data = data[["representations", "attack_cat"]]
data["kfold"] = -1
labelencoder = LabelEncoder()
data["muticlas_label"]=labelencoder.fit_transform(data.attack_cat.values)
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
for fold_no, (t, v) in enumerate(skf.split(data["muticlas_label"], data["muticlas_label"])):
    data.loc[v, "kfold"] = fold_no
data_train = data[data.kfold != 0].reset_index(drop=True)
data_test = data[data.kfold == 0].reset_index(drop=True)
data_train.drop(columns=["kfold", "attack_cat"], inplace=True)
data_test.drop(columns=["kfold", "attack_cat"], inplace=True)
data_train.to_csv(train_data_path, index=False)
data_test.to_csv(test_data_path, index=False)

In [7]:
def transform_payload(x, l, mode):
    x = np.array([int(x[i:i+2],16) for i in range(0, len(x), 2)])
    if mode == "one":
        if len(x) < l:
            return np.pad(x, (0, l-len(x)), 'constant').reshape((1, l))
        elif len(x) == l:
            return x.reshape((1, l))
        else:
            return x[:l].reshape((1, l))
    elif mode == "series":
         if len(x) < l:
            return np.pad(x, (0, l-len(x)), 'constant').reshape((l))
         elif len(x) == l:
            return x.reshape((l))
         else:
            return x[:l].reshape((l))
    else:
        if len(x) < 784:
            return np.pad(x, (0, 784-len(x)), 'constant').reshape((1, 28, 28))
        elif len(x) == 784:
            return x.reshape((1, 28, 28))
        else:
            return x[:784].reshape((1, 28, 28))

In [9]:
test_data = pd.read_csv(test_data_path)
test_payloads = [transform_payload(x, 1500, mode="one") for x in test_data['representations']]
test_data_np = []
test_data_np.append(test_payloads)
test_data_np.append(list(test_data['muticlas_label'].values))
test_data_np = np.array(test_data_np)
np.save(test_data_np_path, test_data_np)

C:\Users\sadantange\AppData\Local\Temp\ipykernel_20212\4246557916.py:15: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  test_data_np = np.array(test_data_np)


In [10]:
train_data_chunks = pd.read_csv(train_data_path, chunksize = 10000000)
train_data_np = []
payload_list = []
label_list = []
for index, chunk in enumerate(train_data_chunks):
    print("Chunk: ", index)
    payloads = [transform_payload(x, 1500, mode='one') for x in chunk['representations']]
    payload_list += payloads
    label_list += list(chunk['muticlas_label'].values)
    del chunk
    gc.collect()
train_data_np.append(payload_list)
train_data_np.append(label_list)
train_data_np = np.array(train_data_np)
np.save(train_data_np_path, train_data_np)

Chunk:  0


C:\Users\sadantange\AppData\Local\Temp\ipykernel_20212\764305729.py:15: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  train_data_np = np.array(train_data_np)


In [8]:
train_data_chunks = pd.read_csv(train_data_path, chunksize = 1000000)
for index, chunk in enumerate(train_data_chunks):
    print("Chunk: ", index)
    train_data_np = []
    payload_list = []
    label_list = []
    payloads = [transform_payload(x, 1500, mode='one') for x in chunk['representations']]
    payload_list += payloads
    label_list += list(chunk['muticlas_label'].values)
    train_data_np.append(payload_list)
    train_data_np.append(label_list)
    train_data_np = np.array(train_data_np)
    chunk_train_data_np_path = train_data_np_path + "_" + str(index)
    np.save(chunk_train_data_np_path, train_data_np)
    del chunk
    gc.collect()

Chunk:  0


C:\Users\sadantange\AppData\Local\Temp\ipykernel_16600\408371658.py:12: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  train_data_np = np.array(train_data_np)


Chunk:  1
Chunk:  2
Chunk:  3
Chunk:  4
Chunk:  5
